In [1]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import numpy as np

from nlp import nmf_model, get_corpus_top_topics
import globals as g
from vectorizer import get_cached_corpus
from pickle_workaround import pickle_load
from vectorizer import tokenize, get_stopwords

from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
from data import get_query_corpus, get_connection
_ = get_connection("/home/mark/Documents/missionmark/missionmark_db_creds")

Connecting to Postgres database...
 -> Connection successful!


<connection object at 0x7fa6609f59c8; dsn: 'user=import_read_user password=xxx dbname=postgres host=analysis.c7znlzygfo4d.us-west-2.rds.amazonaws.com', closed: 0>

In [3]:
query = """SELECT opportunity_id, program_description
           FROM import.govwin_opportunity
           WHERE program_description IS NOT NULL
             AND program_description != ''
        """
doc_ids, corpus = get_query_corpus(query, True)

Loading corpus...
 -> 8847 documents loaded!


In [4]:
tfidf = TfidfVectorizer(min_df=1, max_df=.66, stop_words=get_stopwords(), tokenizer=tokenize, strip_accents="ascii", sublinear_tf=True)
tfidf_corpus = tfidf.fit_transform(corpus)
cv = CountVectorizer(min_df=1, max_df=.66, stop_words=get_stopwords(), tokenizer=tokenize, strip_accents="ascii")
cv_corpus = cv.fit_transform(corpus)

In [6]:
n_topics = 33
nmf = NMF(n_components=n_topics, max_iter=600, random_state=666).fit(tfidf_corpus)

In [7]:
lda = LatentDirichletAllocation(n_components=n_topics, random_state=666, learning_method="online").fit(cv_corpus)
# lda_tfidf = LatentDirichletAllocation(n_components=n_topics, random_state=666, learning_method="online").fit(tfidf_corpus)

In [13]:
pyLDAvis.sklearn.prepare(nmf, tfidf_corpus, tfidf)

/home/mark/ProgramFiles/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
/home/mark/ProgramFiles/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/home/mark/ProgramFiles/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/home/mark/ProgramFiles/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/home/mark/ProgramFiles/anaconda3/lib/pytho

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
13    -0.216837 -0.105389       1        1  6.291290
32    -0.102007 -0.178437       2        1  5.932793
16    -0.214893 -0.035449       3        1  5.690568
14    -0.239352  0.034656       4        1  5.361835
0     -0.104967 -0.065665       5        1  5.072612
22     0.089831  0.097976       6        1  3.976341
31    -0.048597  0.046936       7        1  3.767316
3     -0.037477  0.092950       8        1  3.620307
29    -0.140651  0.067334       9        1  3.547949
5     -0.081756  0.091015      10        1  3.490572
18    -0.087899 -0.092455      11        1  3.326953
11    -0.088586 -0.057406      12        1  3.102238
25    -0.011490  0.055459      13        1  3.050445
24    -0.095326  0.010082      14        1  3.040411
8      0.034961  0.162849      15        1  2.886420
6     -0.157720 -0.004140      16        1  2.785020
23    -0.061626  0.059233      17        1  2.751790
4      0.125145 -0.154182      18        1  2.731597
19     0.158010  0.143282      19        1  2.549160
2      0.094401  0.146981      20        1  2.519698
10    -0.068299 -0.138250      21        1  2.452539
21     0.062278  0.024812      22        1  2.380377
7     -0.009606  0.154705      23        1  2.365111
15    -0.064426 -0.151499      24        1  2.344233
17     0.109319  0.104514      25        1  2.010029
26     0.140323  0.210932      26        1  1.998352
30    -0.007160 -0.002009      27        1  1.862007
27     0.101427  0.111421      28        1  1.740278
12     0.143423  0.101476      29        1  1.601836
28     0.109624 -0.105569      30        1  1.600013
1      0.220822 -0.243001      31        1  1.433979
20     0.219841 -0.212185      32        1  1.378226
9      0.229272 -0.170976      33        1  1.337706, topic_info=      Category        Freq       Term       Total  loglift  logprob
term                                                               
1450   Default  221.000000       armi  221.000000  30.0000  30.0000
5831   Default  176.000000     defens  176.000000  29.0000  29.0000
696    Default  193.000000        air  193.000000  28.0000  28.0000
13690  Default  157.000000    mainten  157.000000  27.0000  27.0000
8729   Default  173.000000       forc  173.000000  26.0000  26.0000
15411  Default  126.000000      naval  126.000000  25.0000  25.0000
9803   Default  110.000000      guard  110.000000  24.0000  24.0000
4485   Default  185.000000    command  185.000000  23.0000  23.0000
6513   Default  126.000000        dod  126.000000  22.0000  22.0000
7635   Default  190.000000      equip  190.000000  21.0000  21.0000
19711  Default  125.000000     repair  125.000000  20.0000  20.0000
6255   Default   95.000000       disa   95.000000  19.0000  19.0000
4734   Default  137.000000  construct  137.000000  18.0000  18.0000
22743  Default  127.000000   supervis  127.000000  17.0000  17.0000
15432  Default  114.000000       navi  114.000000  16.0000  16.0000
20939  Default  160.000000      secur  160.000000  15.0000  15.0000
6371   Default  103.000000   district  103.000000  14.0000  14.0000
4867   Default  116.000000       corp  116.000000  13.0000  13.0000
13897  Default  160.000000     materi  160.000000  12.0000  12.0000
15639  Default  130.000000  necessari  130.000000  11.0000  11.0000
11467  Default   81.000000      inmat   81.000000  10.0000  10.0000
7469   Default  202.000000      engin  202.000000   9.0000   9.0000
10143  Default  126.000000     health  126.000000   8.0000   8.0000
15362  Default   82.000000       nasa   82.000000   7.0000   7.0000
21883  Default  125.000000      space  125.000000   6.0000   6.0000
25587  Default   74.000000       wast   74.000000   5.0000   5.0000
24811  Default   79.000000       usac   79.000000   4.0000   4.0000
627    Default  149.000000      agenc  149.000000   3.0000   3.0000
25213  Default   82.000000    veteran   82.000000   2.0000   2

In [15]:
pyLDAvis.sklearn.prepare(lda, cv_corpus, cv, mds="tsne")

/home/mark/ProgramFiles/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
/home/mark/ProgramFiles/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
18       4.148632   57.083004       1        1  29.726286
20     -31.623707   82.247734       2        1  15.564486
23     -51.620090    4.365107       3        1   9.818048
16     -37.237320   42.777367       4        1   9.315532
19     -72.665085   68.316483       5        1   6.087812
2     -120.049149   71.038727       6        1   3.696828
0      -85.071465  -23.317884       7        1   3.689602
24       3.746644  153.910477       8        1   3.321095
26     -43.892197  125.020538       9        1   3.074668
14     -90.843407   27.547501      10        1   2.390160
4       11.389973  100.666344      11        1   2.172222
27     -96.123108  121.321632      12        1   1.372137
13      -4.638852   14.489150      13        1   1.298029
21      54.059860  119.505707      14        1   1.248220
3        9.546553 -135.647903      15        1   1.124832
6     -129.791428   -9.435646      16        1   1.063520
9       52.523468   60.141781      17        1   0.950730
32     102.787621   70.629639      18        1   0.739615
12     -34.343704 -106.475700      19        1   0.508326
7      -44.003838  -52.889442      20        1   0.422437
25     -85.314468  -85.046524      21        1   0.381578
5      121.905830  -79.129303      22        1   0.377922
11      38.811481   13.003325      23        1   0.315227
30     134.483749    3.974728      24        1   0.263787
1       74.902573 -112.411591      25        1   0.211093
10      86.437561   17.705963      26        1   0.134174
28     103.826744  -33.583233      27        1   0.116891
17      -1.451216  -70.303528      28        1   0.114464
31      30.737801  -38.973602      29        1   0.111946
15      69.086952  -62.379524      30        1   0.105341
22      -6.064906  -27.328667      31        1   0.098648
8       63.749355  -19.157686      32        1   0.097794
29      34.761520  -86.641800      33        1   0.086551, topic_info=      Category          Freq            Term         Total  loglift  logprob
term                                                                        
23903  Default   7809.000000           train   7809.000000  30.0000  30.0000
11467  Default   9263.000000           inmat   9263.000000  29.0000  29.0000
24623  Default  12011.000000            unit  12011.000000  28.0000  28.0000
20939  Default   6926.000000           secur   6926.000000  27.0000  27.0000
5570   Default   9588.000000            data   9588.000000  26.0000  26.0000
19768  Default   4924.000000        research   4924.000000  25.0000  25.0000
10143  Default   6495.000000          health   6495.000000  24.0000  24.0000
1450   Default   5721.000000            armi   5721.000000  23.0000  23.0000
8729   Default   6909.000000            forc   6909.000000  22.0000  22.0000
696    Default   8536.000000             air   8536.000000  21.0000  21.0000
18654  Default   4690.000000         product   4690.000000  20.0000  20.0000
16896  Default  14238.000000            oper  14238.000000  19.0000  19.0000
5831   Default   4307.000000          defens   4307.000000  18.0000  18.0000
8086   Default   7366.000000           facil   7366.000000  17.0000  17.0000
18063  Default   7363.000000            plan   7363.000000  16.0000  16.0000
15386  Default   5071.000000          nation   5071.000000  15.0000  15.0000
8291   Default   4216.000000           feder   4216.000000  14.0000  14.0000
16659  Default   6330.000000           offic   6330.000000  13.0000  13.0000
7469   Default   8333.000000           engin   8333.000000  12.0000  12.0000
23383  Default   7771.000000            test   7771.000000  11.0000  11.0000
11415  Default   8756.000000          inform   8756.000000  10.0000  10.0000
237    Default   4642.000000           activ   4642.000000   9.0000   9.0000
4485   Default   6654.000000         command   6654.000000   8.0000   8.0000
22

In [ ]:
# pyLDAvis.sklearn.prepare(lda_tfidf, tfidf_corpus, tfidf, mds="tsne")